# 1c Active and stative verbs in Leviticus 17-26

In [ ]:
#Dataset path
PATH = "datasets/"

import pandas as pd
import numpy as np
import collections
from IPython.display import clear_output
from tf.app import use

In [ ]:
A = use('bhsa', hoist=globals())

In [ ]:
verbs_qal = '''
book book=Leviticus
 chapter chapter=17|18|19|20|21|22|23|24|25|26
   phrase function=Pred|PreO|PreS|PreC|PtcO
     word pdp=verb vs=qal
'''

verbs_qal = A.search(verbs_qal)

In [ ]:
unique_verbs_qal = set()

for r in verbs_qal:
    unique_verbs_qal.add(F.lex.v(r[3]))
    
len(unique_verbs_qal)

## Compare with active/stative verbs

In [ ]:
data = pd.read_csv(f'{PATH}active_stative_verbs_pca_evaluated.csv')
data = data[['lex','Aktionsart', '0']]
data.columns = ['lex','Aktionsart', 'evaluation']
data.head()

In [ ]:
data['lexemes'] = [r[:r.index('#')] for r in list(data.lex)]

In [ ]:
data.head()

In [ ]:
identified_verbs = set()

for lex in unique_verbs_qal:
    if lex in list(data.lexemes):
        identified_verbs.add(lex)
        
len(identified_verbs)

In [ ]:
#identified_verbs

How many of the verbs are analyzed correctly according to previous manual validation?

In [ ]:
Lev_df = data[data.lexemes.isin(identified_verbs)]

In [ ]:
pd.crosstab(index=Lev_df.evaluation, columns='count')

### Review expected verbs

In [ ]:
Lev_df[Lev_df.evaluation == 'EXP']

### Review cases marked with question-mark

We will explore the annotations marked as ? (unexpected) and review the annotation. The lexemes in question are:

In [ ]:
sur_lex = list(Lev_df[Lev_df.evaluation == '?'].lexemes)
sur_lex

A number of the surprising cases involve so-called transfer verbs, such as "give" (NTN), "take" (LQX), "put" (FJM). Transfer verbs are inherently causative because the agent of the transfer causes the object to be somewhere else, or a patient to become in possesion of the object. Therefore, transfer verbs combine activity and stativity features because there is an active causation and the caused, stative situation. All transferverbs should be marked as stative and causative.

The verbs and their aspect is added to a dictionary for storage:

In [ ]:
Aktionsart_upd = {}

In [ ]:
Aktionsart_upd['LQX['] = 'stative'
Aktionsart_upd['NTN['] = 'stative'

The other cases will be reviewed one by one by means of exploring all instances in Lev 17-26:

In [ ]:
query = '''
book book=Leviticus
 chapter chapter=17|18|19|20|21|22|23|24|25|26
  clause
   phrase function=Pred|PreO|PreS|PreC|PtcO
    word lex={} vs=qal
'''

def display(lex):
    results = A.search(query.format(lex))
    A.show(results)

#### <BD "work, serve"

In [ ]:
display(sur_lex[0])

The first example is difficult because it denotes a causative situation: "You may not have him work as a slave". The other examples are more straight-forward and simply means "to work/serve". However, the most general aspect of this verb is activity.

In [ ]:
Aktionsart_upd['<BD['] = 'active'

#### <BR "pass"

In [ ]:
display(sur_lex[1])

One of the meanings of <BR is to "pass through", that is, an activity. We can explore the verb with adverbial modifiers in the rest of the corpus:

In [ ]:
A.show(A.search(query_corpus.format('<BR[', 'Time'), sets=sets))

We annotate the verb as an active verb.

In [ ]:
Aktionsart_upd['<BR['] = 'active'

#### HLK "walk"

In [ ]:
display(sur_lex[2])

In [ ]:
Aktionsart_upd['HLK['] = 'active'

#### JR> "fear"

In [ ]:
display(sur_lex[3])

In [ ]:
Aktionsart_upd['JR>['] = 'stative'

#### NG< "touch"

In [ ]:
display(sur_lex[5])

In [ ]:
Aktionsart_upd['NG<['] = 'active'

#### NPL "fall"

In [ ]:
display(sur_lex[6])

In [ ]:
Aktionsart_upd['NPL['] = 'stative'

#### RXY "wash"

In [ ]:
display(sur_lex[8])

In [ ]:
Aktionsart_upd['RXY['] = 'stative'

#### XV> "miss"

In [ ]:
display(sur_lex[9])

In [ ]:
Aktionsart_upd['XV>['] = 'stative'

#### ZBX "slaughter"

In [ ]:
display(sur_lex[10])

Both CXV "slaughter" and ZBX "slaughter" are generally used for ritual slaughter. According to Snaith, HRG "kill" is used in non-ritual contexts. Probably all terms imply causation of being dead, although with a focus on the process rather than the induced state of being.

For this reason, we retain CXV as stative and change ZBX into stative:

In [ ]:
Lev_df[Lev_df.lexemes == 'ZBX[']

In [ ]:
#A.show(A.search(query.format('ZBX[')))

In [ ]:
Aktionsart_upd['ZBX['] = 'stative'

### Summary

We have now reviewed all unexpected cases. Some annotations have been retained, while the most annoations have been changed according to closer scrutiny.

We verify that all explored verbs are added to the new dictionary:

In [ ]:
print(f'Missing cases: {len([lex for lex in sur_lex if lex not in Aktionsart_upd])}')

Accordingly, the following updates are made to the dataset:

In [ ]:
new_Aktionsart = [] #We create a list of all Aktionsart - changes or not - and add this list as a new column
no_changes = [] #We collect those cases where the lexeme under investigation has had its Aktionsart changed

for row in Lev_df.iterrows():    
    old_Aktionsart = row[1].Aktionsart
    lex = row[1].lexemes

    if lex in Aktionsart_upd:
        new_Aktionsart.append(Aktionsart_upd[lex])
        
        if old_Aktionsart != Aktionsart_upd[lex]:
            print(f'{lex}: {old_Aktionsart} >> {Aktionsart_upd[lex]}')
            
        else:
            no_changes.append(lex)
    
    else:
        new_Aktionsart.append(old_Aktionsart)
        
print('\n---------------------------------')
print('No changes made:\n')
for lex in no_changes:
    print(lex)

Lev_df['new_Aktionsart'] = new_Aktionsart

In [ ]:
Lev_df.head()

In [ ]:
Lev_df.to_csv(f'{PATH}Lev17-26_Aktionsart_qal_pca_corrected.csv', index=None)

## Annotating remaining *Qal*-verbs

We apt for a simple annotation procedure. Possible annotations for each verb is "stative", "active", or "?" if the verb is difficult. Difficult verbs will be futher scrutinized later but there is no need to spend too much time on relatively straight-forward verbs.

In [ ]:
annotations = 'STA ACT ?'.split()

STOP = 'stop'
error_message = "Input is invalid"

In [ ]:
def input_loop(question, right_answer):
    '''
    input_loop is used when the user is required to make an input. Question has to be a string,
    while right_answer is a list of accepted answer(s).
    '''
    while True:
        test = input(f'{question} ({",".join(right_answer)})') #The user is required to respond to message.
        if test in right_answer:
            return test
        elif test in [STOP]: #Input is accepted
            return test
        else: #Input is wrong and the user has respond again.
            print(error_message)

In [ ]:
def user_input(verbs, query):
    answers = {}
    
    for v in verbs:
        clear_output()
        
        examples = A.search(query.format(v), sets=sets)
        A.show(examples, end=5)
        
        question = f'Aktionsart?'

        Aktionsart = input_loop(question, annotations)
        
        if Aktionsart != STOP:
            
            category = input('Assign catogory (e.g. motion, speech, cognitive, etc.)')
            
            comment = input('Make comment')

        if Aktionsart == STOP:
            return answers
        else:
            answers[v] = Aktionsart, category, comment
    return answers

In [ ]:
prev_runs = []

def getVerbs(orig_data, prev_runs):
   
    annotated_verbs = []
    remaining_verbs = []
         
    ##1. Check previous runs (dictionaries with annotations)
    for r in prev_runs:
        for verb in r:
            annotated_verbs.append(verb)
            
    ##2. Determine the remaining clauses
    for verb in orig_data:
        if not verb in annotated_verbs:
            remaining_verbs.append(verb)
    
    return remaining_verbs
        
verbs = getVerbs(remaining_verbs,prev_runs)

In [ ]:
len(verbs)

In [ ]:
run4 = user_input(verbs)

### Merge outputs

In [ ]:
outputs = [run1, run2, run3, run4]

new_dict = {}
for r in outputs:
    for lex in r:
        new_dict[lex] = r[lex]
        
df = pd.DataFrame(new_dict).T
df.head()

In [ ]:
df.to_csv(f'{PATH}Lev17-26_Aktionsart_qal.csv')

### Reviewing *Aktionsart* of *Qal*-verbs

In [ ]:
data = pd.read_csv(f'{PATH}Lev17-26_Aktionsart_qal.csv')
data.columns = ['lex','Aktionsart','class','comment']
data.head()

We are going to review those verbs that have been question-marked as ambigious cases:

In [ ]:
pd.crosstab(index=data.Aktionsart, columns='count')

In [ ]:
rev_lex = list(data[data.Aktionsart == '?'].lex)
rev_lex.sort()

In [ ]:
#rev_lex

In [ ]:
Aktionsart_upd1 = {}

In [ ]:
corpus = [book for book in F.otype.s('book') if book < T.nodeFromSection(('Isaiah',))]
sets={'corpus':corpus} # make set for searching

query_corpus = '''
corpus
 clause
   phrase function=Pred|PreO|PreS
     word lex={} vs={}
   phrase function={}
'''

def displayCorpus(lex, stem='qal', function='Cmpl|Modi|Time'):
    results = A.search(query_corpus.format(lex, stem, function), sets=sets)
    A.show(results)

#### <CQ "oppress"

In [ ]:
display(rev_lex[0])

In [ ]:
displayCorpus(rev_lex[1],'Cmpl|Time|Loca|Adju|Objc')

In the corpus the verb occurs rarely with modifiers and never with temporal modifiers. It may signal a weakened situation on behalf of the object, a human being, who is being oppressed. Therefore with a new, stative end result.

In [ ]:
Aktionsart_upd1['<CQ['] = 'stative'

#### <ZB "leave"

In [ ]:
display(rev_lex[1])

"leave" can refer to the subject who is to depart or leave, but it can also refer to the object that is to be left. This is the sense in Leviticus. The difference is between "leave" and "leave behind". Here, the verb should be annotated causative stative

In [ ]:
Aktionsart_upd1['<ZB['] = 'stative'

#### >BD "perish"

In [ ]:
display(rev_lex[2])

In [ ]:
displayCorpus(rev_lex[0], 'Cmpl')

The verb >BD occurs several time with modifiers denoting duration ("quicly") and endpoint ("until your are/become perished"). It is probably an accomplishment of a stative verb.

In [ ]:
Aktionsart_upd1['>BD['] = 'stative'

#### FMX "rejoice"

In [ ]:
display(rev_lex[4])

Rejoice denotes an activity that can take place over a course of time at a specific location.

In [ ]:
Lev_df[Lev_df.lexemes == 'FMX[']

In [ ]:
Aktionsart_upd1['FMX['] = 'active'

#### FRP "burn"

In [ ]:
display(rev_lex[5])

"To burn" is an activity. The complement "in/with (ב) fire" denotes the instrument of this activity. Winther-Nielsen analyzes the verb as a consumption verb (active). However, it seems to involve a termination of being - and the Actor (the fire) is not affected by event, therefore stative.

In [ ]:
Aktionsart_upd1['FRP['] = 'stative'

#### GZL "tear away"

In [ ]:
displayCorpus(rev_lex[6], 'Time|Cmpl|Modi')

The verb has the sense of removing, that is, causative of not being at a location.

In [ ]:
Aktionsart_upd1['GZL['] = 'stative'

#### HDR "honour"

In [ ]:
displayCorpus(rev_lex[7], 'Time|Modi|Cmpl')

Given the example in Exodus 23:3 the verb seems to denote something that can happen to the object, in the sense of preference. The example in Leviticus is different, because "to honour" is here a positive case. Nevertheless, the verb may be decomposed as causative of being, independent of this "being" being interpreted positively or negatively:

In [ ]:
Aktionsart_upd1['HDR['] = 'stative'

#### JRC "trample down"

In [ ]:
display(rev_lex[8])

In [ ]:
displayCorpus('JRC[','Time|Cmpl|Loca')

Literally, the verb means "take possesion of" or "inherit", whether it is land or slaves. It is used with locative modifiers describing the geographical extent of domonion. It can also be modifyed by temporal adverbials, such as "for eternity" and "at that time". Perhaps the verb is a causative of possesion because the subject causes someone not to have possesion of the object. However, the examples above does not seem to have this negative meaning. All of them describe a positive view of possesion. It is the subjects who inherits or takes into possession. Moreover, in neither case is the source from which the object is taken, described. Therefore, I would suggest to see the verb as a transfer of possession, the subject causing itself to become into possession of the object.

In [ ]:
Aktionsart_upd1['JRC['] = 'stative'

#### JSP "continue/add"

In [ ]:
display(rev_lex[9])

In Leviticus, the verb JSP generally has two meanings: To continue and to add. These two meanings cannot easily be reconciled in one lexical decomposition. Perhaps the verb should be understood as a causative where the object is the thing to be added or be continued. Therefore stative:

In [ ]:
Aktionsart_upd1['JSP['] = 'stative'

#### LBC "dress"

In [ ]:
display(rev_lex[10])

In [ ]:
displayCorpus(rev_lex[10])

LBC "clothe" carries a notion of both stativity and activity, because one can be clothed for seven days, but one can also put on clothes, an activity with an inherent endgoal. The example from Lev 6:3 is revealing because a locative complement is used "upon his body", that is, the subject "causes a linnentunic to be upon his body". In Leviticus 21:10 the verb is just stative and there is no hint at a causative sense.

In [ ]:
Aktionsart_upd1['LBC['] = 'stative'

#### M>S "reject"

In [ ]:
display(rev_lex[11])

In [ ]:
displayCorpus('M>S[', 'Time|Cmpl|Modi')

The verb can mean "refuse" or "reject", both of which refer to a psychological attitude. In other places it can mean "despise" which is emotional. The verb is probably stative.

In [ ]:
Aktionsart_upd1['M>S['] = 'stative'

#### MY> "find"

In [ ]:
display(rev_lex[12])

"To find" is likely an achievement because there is sudden change of affairs. What about temporal modyfiers in the corpus?

In [ ]:
A.show(A.search(query_corpus.format('MY>[', 'Time'), sets=sets))

1 Sam 29:3 poses an interesting case: "I have not found anything about him from the day he fell until today". This example shows that MY> can be used as expressing a continous inquery or investigation with an implied culmination, namely the sudden change of state (cf. also 1 Sam 29:6, 8). We therefore treat the verb as an activity:

In [ ]:
Aktionsart_upd1['MY>['] = 'active'

#### N>P "commit adultery"

In [ ]:
displayCorpus(rev_lex[13], 'Time|Modi|Cmpl|Objc')

The examples are scarse so it is hard to get a clear sense of the verb. It is probably an active verb denoting an activity that is denounced in the text.

In [ ]:
Aktionsart_upd1['N>P['] = 'active'

#### NQM "avenge"

In [ ]:
display(rev_lex[14])

In [ ]:
displayCorpus(rev_lex[14], 'Time|Modi|Cmpl')

The verb is difficult because it does not occur with many modifiers. However, it may be interpreted as an activity, e.g. "doing revenge".

In [ ]:
Aktionsart_upd1['NQM['] = 'active'

#### NV< "plant"

In [ ]:
display(rev_lex[15])

NT< "plant" denotes an activity but may also assume an aspect of transfer, that is, to cause something to be in the soil. We can explore the verb with its complements by including other cases from the corpus:

In [ ]:
displayCorpus('NV<[', 'Loca|Time|Cmpl')

NV< comes with very few adverbial modifiers. In fact, the verb takes neither time phrases nor complement phrases. There are four examples of NV< with locative phrases.

In [ ]:
Lev_df[Lev_df.lexemes == 'NV<[']

ZR<, on the other hand, seems to be an activity because the object is usually a field:

In [ ]:
Lev_df[Lev_df.lexemes == 'ZR<[']

In [ ]:
Aktionsart_upd1['NV<['] = 'stative'
Aktionsart_upd1['ZR<['] = 'active'

#### PQD "visit"

In [ ]:
display(rev_lex[16])

Curiously, the verb is causative: "I will cause its sin to be upon it". Normally the verb is active, meaning "visit", "instruct", "summon", "call to account". We take the verb as active in the sense of avenging sin.

In [ ]:
Lev_df[Lev_df.lexemes == rev_lex[16]]

In [ ]:
Aktionsart_upd1['PQD['] = 'active'

#### PRC "explain"

In [ ]:
display(rev_lex[17])

In [ ]:
displayCorpus('PRC[', 'Time|Cmpl|Modi|Objc')

The example from Leviticus is the only instance of this verb in the *Qal*. It could be understood as a speech verb with a complement of addresees and the adjunct phrase expressing the theme of the of speech, cf. Num 21:1:

In [ ]:
query1 = '''
clause
  phrase function=Pred
    word lex=DBR[|>MR[|YBH[
  phrase function=Adju
    =: word lex=<L
'''

results = A.search(query1)
A.show(results)

In [ ]:
Aktionsart_upd1['PRC['] = 'active'

#### PRM "rent"

In [ ]:
display(rev_lex[18])

In [ ]:
displayCorpus('PRM[', 'Time|Modi|Cmpl|Adju')

The verb probably means "to tear to pieces" and is in this sense similar to "break", a causative accomplishment, hence stative

In [ ]:
Aktionsart_upd1['PRM['] = 'stative'

#### QRX "make bald"

In [ ]:
displayCorpus(rev_lex[19], 'Time|Modi|Cmpl')

The verb seems to denote an activity, the object being the theme.

In [ ]:
Aktionsart_upd1['QRX['] = 'active'

#### QYR "harvest"

In [ ]:
display(rev_lex[20])

In [ ]:
displayCorpus('QYR=[', 'Cmpl|Time|Loca|Adju')

QYR "to harvest" does not have many modifyers in the corpus. ֹ One can interpret the verb as the activity of cutting grain, but it seems also to imply the notion of gathering the grain because the harvester is supposed to leave some of the harvest in the field.
Therefore, it is a verb of transfer, that is, causing oneself to have the grain.

The same is true of other verbs for harvesting, namely ZMR "gather" and BYR "gather grapes":

In [ ]:
Aktionsart_upd1['QYR=['] = 'stative'
Aktionsart_upd1['ZMR=['] = 'stative'
Aktionsart_upd1['BYR['] = 'stative'

#### RB< "lie down"

In [ ]:
displayCorpus(rev_lex[21], 'Time|Modi|Cmpl|Objc|Adju')

The verb probably denotes an actitivy of some sort.

In [ ]:
Aktionsart_upd1['RB<=['] = 'active'

#### RYH "pay off"

In [ ]:
display(rev_lex[22])

In [ ]:
displayCorpus(rev_lex[7], 'Time|Cmpl|Loca|Adju|Objc')

The temporal adverbials in Lev 26:34 shows that the verb RYH "pay off" describes a situation or event that can occur over a course of time. Curiously, some translations translate the verb differently depending on the subject. The "land" is described as being paid compensation, while the "people" is described as paying off their debt. Other translations choose the more common meaning of the verb in *qal*, namely "enjoy", a stative verb of emotions. The problem is, of course, that this translation does not fit 26:43 where the people are supposed to repay their sins, certainly not enjoy their sins. That case may be an exception. 

In [ ]:
Aktionsart_upd1['RYH=['] = 'stative'

#### YWD "hunt"

In [ ]:
display(rev_lex[23])

In [ ]:
displayCorpus(rev_lex[23], 'Time|Modi|Cmpl')

The verb does not occur with many adverbials. The example from Genesis, however, suggests seeing hunting as some kind of capture and transfer: "Get me some game". Therefore, it seems reasonable to treat the verb as a causative of possession, that is, "to cause someone to have game"

In [ ]:
Aktionsart_upd1['YWD['] = 'stative'

### Summary

We are now ready to summarize the annotations made for all *Qal*-verbs in Leviticus 17-26. First, we verify that all verbs for review have indeed been reviewed:

In [ ]:
print(f'Missing cases: {len([lex for lex in rev_lex if lex not in Aktionsart_upd1])}')

Now, we can add the changes to the dataset:

In [ ]:
data.head()

In [ ]:
new_Aktionsart = [] #We create a list of all Aktionsart - changes or not - and add this list as a new column
no_changes = [] #We collect those cases where the lexeme under investigation has had its Aktionsart changed

for row in data.iterrows():    
    old_Aktionsart = row[1].Aktionsart.lower()
    lex = row[1].lex

    if lex in Aktionsart_upd1:
        new_Aktionsart.append(Aktionsart_upd1[lex][:3])
        
        if old_Aktionsart != Aktionsart_upd1[lex]:
            print(f'{lex}: {old_Aktionsart} >> {Aktionsart_upd1[lex][:3]}')
            
        else:
            no_changes.append(lex)
    
    else:
        new_Aktionsart.append(old_Aktionsart)
        
print('\n---------------------------------')
print('No changes made:\n')
for lex in no_changes:
    print(lex)

data['new_Aktionsart'] = new_Aktionsart

In [ ]:
data = data[['lex','new_Aktionsart','class','comment']]
data.head()

In [ ]:
data.to_csv(f'{PATH}Lev17-26_Aktionsart_qal_corrected.csv', index=None)

## Annotating *Aktionsart* for all other verbs in Lev 17-26

Having dealt with all relevant verbs in the *Qal* we can now expand our focus to verbs in other stem formations. We exclude lexemes that also occur in the *Qal* because we assume that the *Aktionsart* of those verbs can be derived from the *Qal*. Whether this is indeed true will be verified later.

We first extract all relevant lexemes irrespective of stem:

In [ ]:
all_verbs = '''
book book=Leviticus
 chapter chapter=17|18|19|20|21|22|23|24|25|26
   phrase function=Pred|PreO|PreS|PreC|PtcO
     word sp=verb
'''

all_verbs = A.search(all_verbs)

In [ ]:
all_unique_verbs = set()

for r in all_verbs:
    all_unique_verbs.add(F.lex.v(r[3]))
    
len(all_unique_verbs)

We remove those lexemes that have already been annotated:

In [ ]:
#Previous datasets
data_pca = f'{PATH}Lev17-26_Aktionsart_qal_pca_corrected.csv'
data_qal_manual = f'{PATH}Lev17-26_Aktionsart_qal_corrected.csv'

#The lexemes are extracted from each file and added to the same list:
lexemes = list(pd.read_csv(data_pca).lexemes) + list(pd.read_csv(data_qal_manual).lex)
len(lexemes)

In [ ]:
remaining_verbs1 = set()

for lex in all_unique_verbs:
    if lex not in lexemes:
        remaining_verbs1.add(lex)
        
len(remaining_verbs1)

### Annotation

Like for the *Qal* verbs, we opt for the same kind of simple annotation, namely "stative", "active", or "?" if the lexeme is difficult and need further study:

In [ ]:
query1 = '''
book book=Leviticus
 chapter chapter=17|18|19|20|21|22|23|24|25|26
  clause
   phrase function=Pred|PreO|PreS|PreC|PtcO
    word lex={}
'''

In [ ]:
verbs = getVerbs(remaining_verbs1, [run1, run2, run3])

In [ ]:
len(verbs)

In [ ]:
run3 = user_input(verbs, query1)

In [ ]:
outputs = [run1, run2, run3]

new_dict = {}
for r in outputs:
    for lex in r:
        new_dict[lex] = r[lex]
        
df = pd.DataFrame(new_dict).T
df.head()

In [ ]:
df.to_csv(f'{PATH}Lev17-26_Aktionsart_other_stems.csv')

### Reviewing ambigious cases

Like before, we are going to review ambigious cases to complete the analysis of stativity/activity for verbs in Lev 17-26:

In [ ]:
data = pd.read_csv(f'{PATH}Lev17-26_Aktionsart_other_stems.csv')
data.columns = ['lex','Aktionsart','class','comment']
data.Aktionsart = data.Aktionsart.astype('str')

In [ ]:
data.head()

There are 30 cases for review:

In [ ]:
pd.crosstab(index=data.Aktionsart, columns='count')

In [ ]:
Aktionsart_upd2 = {}

In [ ]:
rev_lex = list(data[data.Aktionsart == '?'].lex)
rev_lex.sort()

Basically, there are two types of lexemes to be reviewed:
1. One the hand, some verbs do also occur in the *Qal* in other parts of the corpus and may be derived morphologically as well as semantically from the *Qal* stem. This group of verbs should be reviewed on its own with an eye to the meaning in Leviticus and the *Qal* meaning elsewhere.
2. The other group consists of verbs not occuring in the *Qal*. The meaning in the *Piel*, *Hiphil*, or others, should not - or at least cannot - be derived from a *Qal* derivative and should be treated on its own.

We therefore split the group into these two categories:

In [ ]:
query2 = '''
corpus
  clause
     phrase function=Pred|PreO|PreS|PreC|ptcO
       word sp=verb vs=qal lex={}
'''

Qal_deriv = []
no_Qal_deriv = []

for lex in rev_lex:
    results = A.search(query2.format(lex), sets=sets, silent=True)
    
    if len(results) == 0:
        no_Qal_deriv.append(lex)
    else:
        Qal_deriv.append(lex)

In [ ]:
print(f'Number of lexemes with a possible Qal equivalent: {len(Qal_deriv)}')
print(f'Number of lexemes with no Qal equivalent: {len(no_Qal_deriv)}')

#### Reviewing lexemes with possible *Qal* derivatives

In [ ]:
query = '''
book book=Leviticus
 chapter chapter=17|18|19|20|21|22|23|24|25|26
  clause
   phrase function=Pred|PreO|PreS|PreC|PtcO
    word lex={}
'''

def display(lex):
    results = A.search(query.format(lex))
    A.show(results)

##### CKL "be bereaved of children"

In [ ]:
display(Qal_deriv[0])

In [ ]:
displayCorpus(Qal_deriv[0])

The *Piel* of CKL is semantically derivable from the *Qal* as a factive: "cause someone to be bereaved of children". The verb is therefore inherently stative, while the *Piel* adds a factitive/causative meaning:

In [ ]:
Aktionsart_upd2['CKL['] = ('stative', 'derivable')

##### GLH "uncover"

In [ ]:
display(Qal_deriv[1])

In [ ]:
displayCorpus(Qal_deriv[5])

In [ ]:
displayCorpus(Qal_deriv[5], stem='piel')

The verb is not derivable from the *Qal* meaning. In *Qal* the verb can mean "to go into exile" or "to disappear". In the *Piel*, the verb means to expose or uncover. There are not many complements or other adverbial modifyers to help decomposing the lexeme. It is an activity that does something to the "nakedness" of a womanּ. It is used with "openening someone's eyes", thus it is a causative event, resulting in a stative situation of "being open" or "being exposed" or "being ashamed":

In [ ]:
Aktionsart_upd2['GLH['] = ('stative','not derivable')

##### KPR "cover"

In [ ]:
display(Qal_deriv[2])

Curiously, in the examples in Leviticus both the priest and the blood are presented as subjects of "cover", that is, "to atone". What is the meaning in the *Qal*?

In [ ]:
displayCorpus(Qal_deriv[2])

The *Qal* is also used to describe covering of an object. Therefore, the *Piel* sense is not derivable from the *Qal*. The *Piel* sense, "to make atonement" may simply be interpreted as an activity because we don't have much clue about how the verb can be modified by adverbials.

In [ ]:
Aktionsart_upd2['KPR['] = ('active', 'not derivable')

##### NGP "hurt"

In [ ]:
display(Qal_deriv[3])

In [ ]:
displayCorpus(Qal_deriv[3], stem='nif')

The verb is derivable from the *Qal*, meaning "to hurt" or "to strike/hit". It is apparently punctual, because it can be used with the modifyer "to day", and also "there", describing a precise time and location for the event.

In [ ]:
Aktionsart_upd2['NGP['] = ('active','derivable')

##### RWX "be spacious"

In [ ]:
display(Qal_deriv[4])

In [ ]:
displayCorpus(Qal_deriv[3], function='Cmpl|Objc|Modi|Adju|Time|Loca')

The verb in *Hiphil* does not seem to be derivable from the *Qal* where it apparently means "to amuse" or "to play". In the *Hiphil*, the meaning seems to "enjoy", or perhaps "scent", an activity of directed perception:

In [ ]:
Aktionsart_upd2['RWX['] = ('active', 'not derivable')

##### SLX "forgive"

In [ ]:
display(Qal_deriv[5])

In [ ]:
displayCorpus(Qal_deriv[4], stem='nif')

The verb in *Niphal* is not easialy derivable from the *Qal*, because even in *Niphal* the object to be forgiven is not the subject but the complement, as in the *Qal*. In *Niphal* the verb is stative:

In [ ]:
Aktionsart_upd2['SLX['] = ('stative', 'not derivable')

##### XCB "account"

In [ ]:
display(Qal_deriv[6])

In [ ]:
displayCorpus(Qal_deriv[6])

In [ ]:
displayCorpus(Qal_deriv[6], stem='piel')

To "consider" or "compute" in the *Piel* sense is not easily derivable from the *Qal* meaning as a causative or factitive:

In [ ]:
Aktionsart_upd2['XCB['] = ('active', 'not derivable')

##### XZQ "be strong"

In [ ]:
display(Qal_deriv[7])

The verb has a special meaning in the *Hiphil* where it means "seize" or "grasp". In that case, the verb is an activity, perhaps even an achievement because the change of state takes place in an instant of time.

In [ ]:
Aktionsart_upd2['XZQ['] = ('stative', 'derivable')

#### Review of lexemes with no *Qal* derivative

In [ ]:
Aktionsart_upd3 = {}

##### <LM "hide"

In [ ]:
display(no_Qal_deriv[0])

In [ ]:
displayCorpus(no_Qal_deriv[6], stem='hif')

The example from Leviticus is the most revealing of the examples given since it comes with an emphasizing phrase, an object and a complement. Literaly, the clause could be translated as "causing their eyes not to be on this man", therefore causative stative.

In [ ]:
Aktionsart_upd3['<LM['] = 'stative'

##### <NN "appear"

In [ ]:
display(no_Qal_deriv[1])

In [ ]:
displayCorpus(no_Qal_deriv[7], stem='piel')

The verb can be used as making/bringing a cloud, as in Gen 9:14. In Leviticus, the verb is used as parallel to divine, suggesting that some cultic activity is going on. We can interpret the verb as an activity:

In [ ]:
Aktionsart_upd3['<NN['] = 'active'

##### <RH "pour out"

In [ ]:
displayCorpus(no_Qal_deriv[2], stem='hif|piel', function='Objc')

The verb is difficult because it is very infrequent and is rarely modified by adverbials. In Genesis the verb is apparently used for emptying a bucket (although there is a text-critical issue), but in Leviticus the objects are different. Here, the objects are "well" and "body", neither of them can hardly be thought of as something that can be emptied, except perhaps for the well. We shall probably understand the expression as parallel to the common expression GLH "uncover", because a well and a body can be laid bare or uncovered. Therefore, we decompose the lexeme similar to GLH, that is, causative accomplishment:

In [ ]:
Aktionsart_upd3['<RH['] = 'stative'

##### BQC "search"

In [ ]:
display(no_Qal_deriv[3])

In [ ]:
displayCorpus(no_Qal_deriv[0], stem='piel')

The various examples show that BQC "search" does not neccessarily has an endgoal, since the search can be in vain. 1 Sam 27:1 shows that the verb can be modified by the adverbial עוד "still" which implies duration. In short, the verb is an activity of movement although involving cognition:

In [ ]:
Aktionsart_upd3['BQC['] = 'active'

##### CXT "destroy"

In [ ]:
display(no_Qal_deriv[4])

In [ ]:
displayCorpus(no_Qal_deriv[1], stem='hif')

The example from Judg 20:21 is revealing because includes a quantitative object, a punctual time phrase and a directional spatial phrase. The event is probably stative with a resulting event of extermination, also explaining the case from Leviticus. Therefore causative stative, like other verbs of extinction:

In [ ]:
Aktionsart_upd3['CXT['] = 'stative'

##### DWB "pine away"

In [ ]:
display(no_Qal_deriv[5])

In [ ]:
displayCorpus(no_Qal_deriv[18], stem='hif|hof|piel|pual|nif', function='Objc')

The lexeme is probably causative and describes something that happens to the object, in this case "soul". We can assume that it is a psychological condition. Therefore, causative stative.

In [ ]:
Aktionsart_upd3['DWB['] = 'stative'

##### GLX "shave"

In [ ]:
display(no_Qal_deriv[6])

In [ ]:
displayCorpus(no_Qal_deriv[5], stem='piel')

The examples do not help much in decomposing the verb. Basically, the verb means "shave" and is probably a simple activity

In [ ]:
Aktionsart_upd3['GLX['] = 'active'

##### JDH "praise"

In [ ]:
display(no_Qal_deriv[7])

In [ ]:
displayCorpus(no_Qal_deriv[20], stem='nif|piel|pual|hif|hof', function='Objc|Cmpl|Modi|Time')

The verb usually refers to "praising God", a communicative situation where the object is the one being praised and not the actual performance. In Leviticus, the lexeme refers to a confession and the object is not the addressee but the content of the confession. We can therefore treat the lexeme as active:

In [ ]:
Aktionsart_upd3['JDH['] = 'active'

##### JKX "reprove"

In [ ]:
display(no_Qal_deriv[8])

In [ ]:
displayCorpus(no_Qal_deriv[19], stem='nif|hif|hof|piel|pual')

The verb has many meanings but is commonly used for physical or verbal discplin/chastice. It can also refer to an argument between two parties. It refers to a communicative or social activity where tools can be used as instruments. Therefore, an an activity:

In [ ]:
Aktionsart_upd3['JKX['] = 'active'

##### JSR "admonish"

In [ ]:
displayCorpus(no_Qal_deriv[9], stem='nif|piel|pual|hif|hof')

The verb occurs both in *Niphal* and in *Piel*, the former referring to the cognitive status of the subject, apparently "learn" or "let oneself be instructed" (cf. HALOT). The Piel may be a causative use, that is, causing someone to undergo a cognitive process; therefore causative accomplishment (stative).

NB: Because the *Piel* can reasonably be derived from the *Niphal* we add it to Aktionsart_upd2 which deals with derivable relationships between stems:

In [ ]:
Aktionsart_upd2['JSR['] = ('stative', 'derivable')

##### KSH "cover"

In [ ]:
display(no_Qal_deriv[10])

In [ ]:
displayCorpus(no_Qal_deriv[9], stem='piel|pual')

The basic meaning of the verb is "to cover something with something", that is, the verb often occurs with an object to be covered but also an instrument by which the object is covered. We can therefore treat the verb as inherently causative and also stative because the event of covering causes a stative situation that can last "for seven days".

In [ ]:
Aktionsart_upd3['KSH['] = 'stative'

##### KXC "grow lean"

In [ ]:
display(no_Qal_deriv[11])

In [ ]:
displayCorpus(no_Qal_deriv[4], stem='piel')

The verb in *Piel* commonly means to keep secret or to deny or lie, the preposition starting with B expressing the theme to be kept secret or lied about. We can reasonably treat the verb as a communicative event, therefore active: 

In [ ]:
Aktionsart_upd3['KXC['] = 'active'

##### MQQ "putrefy"

In [ ]:
displayCorpus(no_Qal_deriv[14], stem='nif|piel|pual|hif|hof|hitp', function='Objc|Cmpl|Modi|Time')

The verb only occurs in Leviticus in this corpus. However, it also occurs in the prophetic and poetic literature of the Hebrew Bible with the meaning "melt/decay/rot". Therefore, we may treat the the verb as an accomplishment of a stative verb; in Leviticus describing a psychological process.

In [ ]:
Aktionsart_upd3['MQQ['] = 'stative'

##### NFG "overtake"

In [ ]:
display(no_Qal_deriv[13])

In Leviticus the verb is used in two different ways, that is, 1) to afford, and 2) to reach/extend. We can explore the verb in the larger corpus:

In [ ]:
displayCorpus(no_Qal_deriv[8], stem='hif|hof|piel|nif')

The corpus is not of much help. Implied in the verb, however, seems to be a notion of endgoal, something to be achieved or reached, perhaps over a course of time. Therefore stative in the meaning "be sufficient/enough".

In [ ]:
Aktionsart_upd3['NFG['] = 'stative'

##### NQP "go around"

In [ ]:
displayCorpus(no_Qal_deriv[12], stem='piel|pual|hif|hof', function='Objc|Cmpl|Modi|Time')

While the lexeme is normally used with the verb SBB "go around" to express an encircling or surrounding of a city, the lexeme is used rather differently in Leviticus, as it means to trim the hair. The normal aspect is therefore an active accomplishment.

In [ ]:
Aktionsart_upd3['NQP['] = 'active'

##### NXC "divine"

In [ ]:
display(no_Qal_deriv[17])

In [ ]:
displayCorpus(no_Qal_deriv[17], stem='piel|pual')

The verb is difficult to parse because it occurs infrequently and without any temporal indicators. However, it occurs with an intensive modifier, suggesting that the verb is active:

In [ ]:
Aktionsart_upd3['NXC['] = 'active'

##### NYH "decay"

In [ ]:
display(no_Qal_deriv[16])

In [ ]:
displayCorpus(no_Qal_deriv[16], stem='nif|piel|pual|hif|hof', function='Objc|Cmpl|Modi|Time')

The verb is used to express a violent fighting. For that reason, it is curious that the verb occurs in the *Niphal* which is regularly used to express passive. Perhaps, *Niphal* here expresses the reciprocal sense because the fight necessarily involves two parties. The verb is active:

In [ ]:
Aktionsart_upd3['NYH['] = 'active'

##### NZR "dedicate"

In [ ]:
display(no_Qal_deriv[17])

In [ ]:
displayCorpus(no_Qal_deriv[3], stem='hif')

The basic meaning of NZR is to "restrain" or withdraw from a certain practice. The meaning of the verb in Leviticus is dubious. Perhaps it denotes an attitude, or perhaps it means to refrain from eating the gifts of the Israelites. One possible interpreation is simply to "be away from", also accounting for one of the meanings in *Hiphil*, namely to cause others to be away from... The stative would also support an interpretation of NZR as denoting an attitude (stative).

In [ ]:
Aktionsart_upd3['NZR['] = 'stative'

##### PL> "be miraculous"

In [ ]:
display(no_Qal_deriv[18])

In [ ]:
displayCorpus(no_Qal_deriv[21], stem='piel', function='Objc')

In the *Piel* the verb is used in conjunction with "vow" and refers to fulfilling the vow by the means of a sacrifice. It is certainly causative because it is the object that becomes fulfilled. Therefore causative stative:

In [ ]:
Aktionsart_upd3['PL>['] = 'stative'

##### QVR "burn"

In [ ]:
display(no_Qal_deriv[19])

In [ ]:
displayCorpus(no_Qal_deriv[2], stem='hif')

The verb is often modified by a locative phrase describing the place of the event as well as a benefactor to whom the sacrifice is addressed. We may deduce that the verb is inherently causative, the subject causing the object to go to the benefactor by smoke. Therefore causative stative: "to cause an object to be/belong"

In [ ]:
Aktionsart_upd3['QVR['] = 'stative'

##### RJG "be empty"

In [ ]:
displayCorpus(no_Qal_deriv[13], stem='hif|hof', function='Objc|Cmpl|Modi|Time')

The verb is exclusively used in a military context, two of the three times with "sword" as object. The Genesis text is disputed. The verb may be derived from the adjective RJQ "empty", so it should probably be translated as "drawing my sword". The complement "after them" suggests a direction of the sword, that is "cause the sword to go after them"; therefore, causative active accomplishment. This interpretation is supported by the fact that the >XR "after" usually appears with verbs of motion:

In [ ]:
AXR = '''
corpus
 clause
  phrase function=Pred|PreO|PreS
    word
  phrase function=Cmpl
    =: word lex=>XR/
'''

AXR = A.search(AXR, sets=sets)

verb_list = []
for r in AXR:
    verb = F.lex.v(r[3])
    verb_list.append(verb)
    
pd.DataFrame([collections.Counter(verb_list)]).T.sort_values(by=0, ascending=False)[:10]

In [ ]:
Aktionsart_upd3['RJQ['] = 'active'

##### XRP "spend autumn"

In [ ]:
display(no_Qal_deriv[21])

In [ ]:
displayCorpus(no_Qal_deriv[10], stem='nif|piel|pual|hif|hof', function='Cmpl|Objc')

The verb is a *hapax* and is obviously difficult to parse. It has been suggested that the verb means "be engaged"; therefore stative

In [ ]:
Aktionsart_upd3['XRP=['] = 'stative'

#### Summary

To check whether all lexemes have been reviewed, we collocate the two lists of lexemes (with and without possible *Qal*-
derivaties) and check whether they occur in the two dictionaries:

In [ ]:
combined_lex = Qal_deriv + no_Qal_deriv

In [ ]:
print(f'Missing cases: {len([lex for lex in combined_lex if lex not in Aktionsart_upd2 and lex not in Aktionsart_upd3])}')

In [ ]:
data.head()

In [ ]:
new_Aktionsart = [] #We create a list of all Aktionsart - changes or not - and add this list as a new column
no_changes = [] #We collect those cases where the lexeme under investigation has had its Aktionsart changed

for row in data.iterrows():    
    old_Aktionsart = row[1].Aktionsart.lower()
    lex = row[1].lex

    if lex in Aktionsart_upd2:       
        if old_Aktionsart != Aktionsart_upd2[lex]:
            
            if type(Aktionsart_upd2[lex]) != 'str':
                new_Aktionsart.append(Aktionsart_upd2[lex][0][:3])
                print(f'{lex}: {old_Aktionsart} >> {Aktionsart_upd2[lex][0][:3]}')
            
            else:
                new_Aktionsart.append(Aktionsart_upd2[lex][:3])
                print(f'{lex}: {old_Aktionsart} >> {Aktionsart_upd2[lex][:3]}')
            
        else:
            no_changes.append(lex)
            
    elif lex in Aktionsart_upd3:
        new_Aktionsart.append(Aktionsart_upd3[lex][:3])
        
        if old_Aktionsart != Aktionsart_upd3[lex]:
            print(f'{lex}: {old_Aktionsart} >> {Aktionsart_upd3[lex][:3]}')
            
        else:
            no_changes.append(lex)
    
    else:
        new_Aktionsart.append(old_Aktionsart)
        
print('\n---------------------------------')
print('No changes made:\n')
for lex in no_changes:
    print(lex)

data['new_Aktionsart'] = new_Aktionsart

In [ ]:
data.head()

In [ ]:
data_upd = data[['lex','new_Aktionsart','class','comment']]

In [ ]:
data_upd.to_csv(f'{PATH}Lev17-26_Aktionsart_other_stems_corrected.csv', index=None)

## Prepare data for CSV-export

Because lexemes may be used to express several meanings, it is pointless to assign *Aktionsart* to the glosses, as if one lexeme had only one meaning irrespective of stem.

For that reason the *Aktionsart* is assigned to concrete word nodes in the corpus based on two parameters, lex and stem. Some lexemes may have different meanings even with the same stem, and the only solution will be to manually correct those cases. For the most part, lexemes generally have the same meaning in the same stem and some meanings in non-*Qal* stems can even be derived from the *Qal* correlate.

First, we collect all data according to lex and stem. The two first files contain only information about lexemes in the *Qal*. We collocate lexeme and stem to be able to disambiguate word senses according to this combination:

In [ ]:
file1 = f'{PATH}Lev17-26_Aktionsart_qal_pca_corrected.csv'
file2 = f'{PATH}Lev17-26_Aktionsart_qal_corrected.csv'
file3 = f'{PATH}Lev17-26_Aktionsart_other_stems_corrected.csv'

In [ ]:
#Extracting data from data1:
data1 = pd.read_csv(file1)
data1.new_Aktionsart = [n[:3] for n in list(data1.new_Aktionsart)]
data1['lex'] = [n+'_qal' for n in list(data1.lexemes)]
data1 = data1[['lex','new_Aktionsart']]

#Extracting data from data2:
data2 = pd.read_csv(file2)
data2 = data2[['lex','new_Aktionsart']]
data2['lex'] = [n+'_qal' for n in list(data2.lex)]

The third file is more difficult because it contains the *Aktionsart* of lexemes in non-*Qal* stems.

In [ ]:
data3 = pd.read_csv(file3)
data3 = data3[['lex','new_Aktionsart']]
data3

In [ ]:
non_qal_lex = list(data3.lex)

verbs_non_qal = '''
book book=Leviticus
 chapter chapter=17|18|19|20|21|22|23|24|25|26
   phrase function=Pred|PreO|PreS|PreC|PtcO
     word sp=verb lex={}
'''

verbs_non_qal = A.search(verbs_non_qal.format('|'.join(non_qal_lex)))

In [ ]:
lex_stem = set()

for r in verbs_non_qal:
    lex = F.lex.v(r[3])
    stem = F.vs.v(r[3])
    lex_stem.add(f'{lex}_{stem}')
    
print(f'{len(lex_stem)} lexeme-stem combinations found')

In [ ]:
lex_stem_dict = {}

for c in lex_stem:
    lex = c[:c.index('_')] #Isolating lexeme from lex_stem combination
    
    Aktionsart = data3[data3.lex == lex].new_Aktionsart.item()
    lex_stem_dict[c] = Aktionsart
    
data3a = pd.DataFrame([lex_stem_dict]).T
data3a['lex'] = data3a.index
data3a.columns = ['new_Aktionsart','lex']

In [ ]:
data3a = data3a[['lex','new_Aktionsart']]
data3a.head()

In [ ]:
comb_df = pd.concat([data1, data2, data3a])
comb_df.head()

In [ ]:
comb_df[comb_df.lex=='GRC[_qal']

### Combining lexemes, stems and annotation for CSC-export

In [ ]:
comb_df['stem'] = [n[n.index('_')+1:] for n in list(comb_df.lex)]
comb_df['lex'] = [n[:n.index('_')] for n in list(comb_df.lex)]

In [ ]:
Aktionsart_upd2

The procedure is three-fold:
1. If the lexeme is in the *Qal*-stem, the other stems of the lexemes are annotated with the *Aktionsart*
2. If the lexeme is not in the *Qal*-stem but is annotated as derivable in Aktionsart_upd2, then the *Qal* stem is also
annotated with the *Aktionsart*
3. If the lexeme is not in the *Qal*-stem and not derivable, then only this particular stem will be annotated

In [ ]:
comb_df[comb_df.lex == 'GRC[']

In [ ]:
Aktionsart_dict = {}

lexemes = list(comb_df.lex)

last_word = L.d(T.nodeFromSection(('2_Kings',25,30)), 'word')[-1]

for n in range(1,last_word+1):
    if F.pdp.v(n) == 'verb' and F.function.v(L.u(n, 'phrase')[0]) in ['Pred','PreS','PreO','PreC','PtcO']:
        lex = F.lex.v(n)
        stem = F.vs.v(n)
        
        if lex in lexemes:
            
            df_sub = comb_df[comb_df.lex==lex]
            stems = list(df_sub.stem)
            
            #1. If Qal
            if 'qal' in stems:
                Aktionsart_dict[f'{lex}_{stem}'] = df_sub.new_Aktionsart.item()
                
            #2. If not Qal but derivable
            else:
                if lex in Aktionsart_upd2 and Aktionsart_upd2[lex][1] == 'derivable':
                    stems.append('qal')
                    
                    if stem in stems:
                        Aktionsart_dict[f'{lex}_{stem}'] = df_sub[df_sub.stem==stems[0]].new_Aktionsart.item()
                        
                elif lex in Aktionsart_upd2 and Aktionsart_upd2[lex][1] != 'derivable':
                    if stem in stems:
                        Aktionsart_dict[f'{lex}_{stem}'] = df_sub[df_sub.stem==stem].new_Aktionsart.item()
                        
                #3. If not Qal and not derivable
                elif stem in stems:
                    Aktionsart_dict[f'{lex}_{stem}'] = df_sub[df_sub.stem==stem].new_Aktionsart.item()

In [ ]:
Aktionsart_df = pd.DataFrame([Aktionsart_dict]).T
Aktionsart_df.head()

In [ ]:
Aktionsart_df.to_csv(f'{PATH}Lev17-26.Aktionsart_sta_act_final_1.csv')

#### ͏͏͏Final check:

In [ ]:
data1 = pd.read_csv(f'{PATH}Lev17-26.Aktionsart_sta_act_final.csv')
data2 = pd.read_csv(f'{PATH}Lev17-26.Aktionsart_sta_act_final_1.csv')

data1.columns = ['lex','Aktionsart']
data2.columns = ['lex','Aktionsart']

In [ ]:
data1.head()

In [ ]:
data2.head()

In [ ]:
for n, row in data1.iterrows():
    lex = row.lex
    Aktionsart = row.Aktionsart
    
    Aktionsart2 = data2[data2.lex == lex].Aktionsart.item()
    
    if not Aktionsart == Aktionsart2:
        print(lex, Aktionsart, Aktionsart2)